# Playground for strokes graph

In [98]:
import geopandas as gpd
import matplotlib.pyplot as plt
import momepy
import networkx as nx
import folium
from itertools import combinations
from collections import Counter
import shapely

In [99]:
streets = gpd.read_file(momepy.datasets.get_path("bubenec"), layer="streets")

In [100]:
# Clean data
streets = momepy.remove_false_nodes(streets)
# Add fixed ID to each edge
streets["id"] = streets.index
# Transform into primal graph
G_primal = momepy.gdf_to_nx(streets, approach="primal")
points_primal, lines_primal = momepy.nx_to_gdf(G_primal, points=True, lines=True)
# Use COINS on primal graph edges
coins = momepy.COINS(lines_primal)
# List the stroke for each edge
stroke_attribute = coins.stroke_attribute()
# List each edge for each stroke
stroke_gdf = coins.stroke_gdf()
stroke_gdf["stroke_id"] = stroke_gdf.index
stroke_gdf["edge_ids"] = stroke_gdf.stroke_id.apply(lambda x: list(lines_primal.iloc[stroke_attribute[stroke_attribute == x].index]["id"]))
# Dictionary mapping to each edge ID the edge index
d = {val:lines_primal[lines_primal["id"] == val].index.values[0] for val in lines_primal["id"]}
# Add stroke ID to each edge
nx.set_edge_attributes(G_primal, {e: int(stroke_attribute[d[G_primal.edges[e]["id"]]]) for e in G_primal.edges}, "stroke_id")


In [101]:
# m = stroke_gdf.explore(tiles="cartodb-positron", column="stroke_id", cmap="tab10", name="strokes")
# lines_primal.explore(m=m, column="id", name = "lines", cmap="viridis")
# folium.LayerControl().add_to(m)
# m

In [102]:
# points, lines = momepy.nx_to_gdf(G_primal, points=True, lines=True)
# m = stroke_gdf.explore(tiles="cartodb-positron", column="stroke_id", cmap="Set2", name="strokes")
# lines.explore(m=m, column="stroke_id", name = "lines", cmap="Set2")
# folium.LayerControl().add_to(m)
# m

In [ ]:
G_dual = nx.Graph()
G_dual.graph["crs"] = G_primal.graph["crs"]
# Create a node for each stroke with the right features
G_dual.add_nodes_from([[val, {attr:(list(stroke_gdf[stroke_gdf["stroke_id"] == val][attr])[0] if attr != "geometry" else stroke_gdf[stroke_gdf["stroke_id"] == val].geometry[val]) for attr in list(stroke_gdf)}] for val in list(stroke_gdf["stroke_id"])])
# For all node, put its geometry at the center of the LineString
for n in G_dual.nodes:
    G_dual.nodes[n]["geometry"] = stroke_gdf.iloc[n].geometry.interpolate(0.5, normalized=True)
    G_dual.nodes[n]["x"] = G_dual.nodes[n]["geometry"].xy[0]
    G_dual.nodes[n]["y"] = G_dual.nodes[n]["geometry"].xy[1]
# Add an edge between the strokes that are intersectin in at least one node from the primal graph
for n in G_primal.nodes:
    # Check the number of strokes at the node
    strokes_present = [v for _, _, v in G_primal.edges(n, data="stroke_id", keys=False)]
    if len(set(strokes_present)) > 1:
        # If more than one, check all the pairs of strokes possible
        pairs = list(combinations(set(strokes_present), 2))
        for p in pairs:
            # Count the occurences of the strokes
            strokes_counting = [i for i in strokes_present if i in p]
            # If there is no edge between the strokes, add one
            if not G_dual.has_edge(p[0], p[1]):
                G_dual.add_edge(p[0], p[1], geometry=shapely.LineString([G_dual.nodes[p[0]]["geometry"], G_dual.nodes[p[1]]["geometry"]]), counter=Counter(strokes_counting))
            else:
                G_dual.edges[p[0], p[1]]["counter"].update(strokes_counting)

In [130]:
points_dual, lines_dual = momepy.nx_to_gdf(G_dual, points=True, lines=True)
m = stroke_gdf.explore(tiles="cartodb-positron", column="stroke_id", cmap="tab10", name="strokes")
points_primal.explore(m=m, name="points_primal")
lines_primal.explore(m=m, name="lines_primal")
points_dual.explore(m=m, color="black", marker_kwds={"radius":10}, name="points_dual")
lines_dual.explore(m=m, color="blue", name="lines_dual")
folium.LayerControl().add_to(m)
m

/var/folders/mb/_ysy1pzs13qgnh9b942_7lkh0000gn/T/ipykernel_72363/343906018.py:1: UserWarning: Approach is not set. Defaulting to 'primal'.
  points_dual, lines_dual = momepy.nx_to_gdf(G_dual, points=True, lines=True)
